In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import genfromtxt
import time
import math
import random

In [3]:
def pathLength(pathList):
    totalDistance = 0
    firstPoint_x = pathList[0,0]
    firstPoint_y = pathList[0,1]
    previousCity_x = firstPoint_x
    previousCity_y = firstPoint_y
    
    for partialDist in pathList:  
        currentCity_x = partialDist[0]
        currentCity_y = partialDist[1]
        totalDistance = totalDistance + math.sqrt((previousCity_x-currentCity_x)**2 + (previousCity_y-currentCity_y)**2) 
        previousCity_x = currentCity_x
        previousCity_y = currentCity_y            
                    
    totalDistance = totalDistance + math.sqrt((firstPoint_x-currentCity_x)**2 + (firstPoint_y-currentCity_y)**2) #np.linalg.norm(previousCity-firstPoint) #loops back around to the first point
    return totalDistance

In [25]:
cities_15 = genfromtxt('15cities.csv', delimiter=',')
cities_25 = genfromtxt('25cities.csv', delimiter=',')
cities_25A = genfromtxt('25cities_A.csv', delimiter=',')
cities_100 = genfromtxt('100cities.csv', delimiter=',')
testCities = genfromtxt('testcities.csv', delimiter=',')
testCities3 = genfromtxt('testcities3x3.csv', delimiter=',')

np.random.seed() #seed the random generator

#for i in range(10):
#simulatedAnnealing(cities_25A,100)
#randomPopulation(cities_15,3)

In [5]:
def randomSwap(pathList):
    randomPathIndex = np.random.choice(pathList.shape[0],2, replace=False)#picks 2 indicies at random, does not allow the same index to be called twice
    pathList[randomPathIndex[0],:], pathList[randomPathIndex[1],:] = pathList[randomPathIndex[1],:], pathList[randomPathIndex[0],:].copy()
    return pathList

In [27]:
def simulatedAnnealing(citiesList, temperature):
    count = 2 #use this to update T
    solutionsGenerated = 1 #number of soultions generated, updates each time a new path is chosen
    startTime = time.time() #used to keep track opf run time
    T = temperature #temperature variable, may change over time
    bestPath = citiesList[1:,:] #just removes the first row since that is not a city
    np.random.shuffle(bestPath) # creates a random order of cities
    bestPathLength = pathLength(bestPath) #saves the current best path length, initial this is the first random path
    while T > 1:
        pathTrial = randomSwap(bestPath) #conducts the city swap on the current best path
        newPathLength = pathLength(pathTrial) #calcutates the new path length of the random swap
        if newPathLength < bestPathLength:
            #print(newPathLength)
            #print(bestPathLength)
            bestPath = pathTrial
            bestPathLength = newPathLength
            solutionsGenerated += 1
            T *= 0.99
            #tspPlot(bestPath)
        elif newPathLength > bestPathLength:
            pickingProb = np.exp(-(newPathLength-bestPathLength)/ T)
            if pickingProb >= np.random.uniform(0.0,1.0):
                bestPath = pathTrial
                bestPathLength = newPathLength
                solutionsGenerated += 1
                T *= 0.99
                #print("worst")
                #tspPlot(bestPath)
    
        count +=1

        if count > 2000000:
            break
                
    totalTime = time.time() - startTime
    return bestPathLength, totalTime, solutionsGenerated, count

simulatedAnnealing(cities_15,100)

(6257.967702162693, 105.49319219589233, 27, 2000001)

In [6]:
def tspPlot(bestPath):
    
    x = bestPath[:,0]
    y = bestPath[:,1]
    
    plt.plot(x,y, 'co')
    
    a_scale = float(max(x))/float(50)
    
    plt.arrow(x[-1],y[-1], (x[0] - x[-1]), (y[0] - y[-1]), head_width = a_scale, color ='r', length_includes_head = True)
    
    for i in range(0,len(x)-1):
        plt.arrow(x[i],y[i], (x[i+1] - x[i]), (y[i+1] - y[i]), head_width = a_scale, color ='g', length_includes_head = True)
    
    plt.xlim(-1,max(x)*1.1)
    plt.ylim(-1,max(y)*1.1)  
    plt.show()


In [123]:
def evolutionAlgo(citiesList,numPop):  
    
    startTime = time.time() #used to keep track opf run time
    solutionsGenerated = 1
    bestPath = citiesList[1:,:] #just removes the first row since that is not a city
    #for paths in bestPath:        
    np.random.shuffle(bestPath) # creates a random order of cities
    bestPathLength = pathLength(bestPath) #saves the current best path length, initial this is the first random path
    
    nestedPopulations = []  
    
    for i in range(numPop): #generate random populations
        j = 0     
        mixingPopulation = bestPath
        np.random.shuffle(mixingPopulation)
        while j < mixingPopulation.shape[0]*numPop:
            mixingPopulation = randomSwap(mixingPopulation).copy()
            j += 1
        nestedPopulations.append(mixingPopulation.copy())
    
    count = 0
    bestSolutionProb = 0
    desiredProb = 1.1
    while bestSolutionProb < desiredProb:

        #find the fitness of current populstions
        probDistList, probOfPopulation, bestSolutionProb, Fitness = fitness(nestedPopulations,numPop, desiredProb)
        #pick the winner
        #pickedWinner = nestedPopulations
        originalPop = nestedPopulations
        pickingProb = np.random.uniform(0.0,1.0) 

        for i in range(numPop):
            if i == 0:
                if pickingProb <= probDistList[i]:
                    #take the winner and mutate it but dont add it back to the pool until its compared                    
                    #need new fitness                 
                    for i in range(numPop):
                        originalFitness = pathLength(nestedPopulations[i])
                        pickedWinner = randomSwap(nestedPopulations[i].copy())
                        pickedWinnerFitness = pathLength(pickedWinner)
                        difference = pickedWinnerFitness-originalFitness
                        #print(difference,difference<0)
                        if(difference < 0):
                            nestedPopulations[i] = pickedWinner
                            #tspPlot(nestedPopulations[i])
                            bestPathLength = pathLength(nestedPopulations[i])
                            print(bestPathLength)
                            solutionsGenerated += 1

                          
            """else:
                if pickingProb > probDistList[i-1] and pickingProb <= probDistList[i]:
                    #take the winner and mutate it but dont add it back to the pool until its compared
                    pickedWinner[i] = randomSwap(nestedPopulations[i]).copy()
                    #append to list
                    #need new fitness
                    tempDistList, tempProPPop, tempBest, tempFitness = fitness(pickedWinner,numPop, desiredProb)
                    pickedWinnerFitness = tempFitness[i]
                    for i in range(numPop):
                        if(pickedWinnerFitness > Fitness[i]):
                            nestedPopulations[i] = pickedWinner[i]
                            #tspPlot(nestedPopulations[i])
                            bestPathLength = pathLength(nestedPopulations[i])
                            solutionsGenerated += 1
                            break"""
        
        count +=1
        #print(count)
        if count > 50000:
            print(max(probOfPopulation))
            break

    totalTime = time.time() - startTime
    
    return totalTime, bestPathLength, solutionsGenerated, count
    #for i in range(numPop):
    #    tspPlot(nestedPopulations[i])

evolutionAlgo(cities_15,1) 

#evolutionAlgo(testCities3,3)

264.52926121581913 False
901.54772307643 False
-788.1779721899802 True
237.27805152820292 False
1159.5247112171528 False
801.3646529686084 False
349.5843857263644 False
59.47809935558689 False
227.27849686814807 False
1079.7437959797535 False
1915.2115626721843 False
-1044.0011378800918 True
185.92664704708477 False
-240.79990421932962 True
-252.62996074864895 True
728.2585280138246 False
2250.172949884618 False
-217.18378177624072 True
194.9300808443786 False
1061.6060168766526 False
2471.9872078586977 False
1613.098920985647 False
-17.67454614511371 True
469.3492217094936 False
8.079984288051492 False
1265.7458726629702 False
2395.9559279748973 False
17.67454614511371 False
1171.0178293720328 False
2412.341787934285 False
1201.7051639618057 False
983.3067805738119 False
1959.5995215804087 False
1613.098920985647 False
8.079984288051492 False
2749.508346433129 False
35.98002129690758 False
59.113476292346604 False
-83.10161996892748 True
855.9485010646367 False
289.643700112445 False


1316.8812456948535 False
1237.109515047071 False
894.3724321623167 False
184.82929177822825 False
1316.8812456948535 False
1208.6842470016127 False
1239.0598643520234 False
245.32384421974166 False
184.82929177822825 False
901.6332809629976 False
1391.2868585218703 False
1412.5004099985517 False
1060.713977222069 False
292.1923944946375 False
485.55764138421455 False
109.26520918770257 False
2895.6183339852605 False
742.0888822934776 False
742.0888822934776 False
196.96414390253085 False
2018.4212727523745 False
931.9208789961804 False
2685.925311341095 False
402.7582379740643 False
1029.1479501357408 False
2685.925311341095 False
109.26520918770257 False
680.1009865305004 False
1532.510628430212 False
184.82929177822825 False
2014.0663522072982 False
421.63708332412625 False
14.328042762917903 False
5406.096792260173 False
2767.863526374832 False
1252.5292000022473 False
184.82929177822825 False
1917.649669494901 False
523.7318775970134 False
1868.264564236898 False
2012.6714850148664

598.4292101587325 False
2012.6714850148664 False
299.6030535111722 False
3035.88965047088 False
2012.6714850148664 False
1917.649669494901 False
1747.717408937834 False
1239.0598643520234 False
1564.3989218075176 False
3188.263880404432 False
196.96414390253085 False
292.1923944946375 False
1726.169928784403 False
3624.470471482032 False
165.42564264373596 False
525.4344343253088 False
611.8094140481317 False
4533.288840933227 False
1736.2065713298334 False
4533.288840933227 False
464.75582027084783 False
1060.713977222069 False
2379.719980352572 False
464.75582027084783 False
1868.264564236898 False
1237.109515047071 False
402.7582379740643 False
421.63708332412625 False
2671.4617301362678 False
3459.736460259038 False
1560.6965072306311 False
894.3724321623167 False
2057.8506181061102 False
2454.220754742314 False
525.4344343253088 False
1165.4402145426875 False
790.063758270866 False
2012.6714850148664 False
9.52446486340341 False
1412.5004099985517 False
1165.082123274714 False
303

2101.860802740831 False
14.328042762917903 False
2404.0030262097616 False
523.7318775970134 False
1738.6197434441865 False
3719.5736030303297 False
896.360028593097 False
2740.1719613021514 False
1917.649669494901 False
464.75582027084783 False
688.8715080657448 False
2895.6183339852605 False
293.0967172906685 False
680.1009865305004 False
2055.98121700347 False
790.063758270866 False
485.55764138421455 False
1040.4464338513717 False
5145.1506739120605 False
742.0888822934776 False
3188.263880404432 False
1316.8812456948535 False
4533.288840933227 False
1736.2065713298334 False
1060.713977222069 False
2057.8506181061102 False
1560.6965072306311 False
3719.5736030303297 False
4533.288840933227 False
1680.3438469078365 False
1560.6965072306311 False
3459.688932513528 False
1866.8238945426265 False
3375.621421088642 False
222.11236202539658 False
2769.0691469505136 False
598.4292101587325 False
2012.6714850148664 False
165.42564264373596 False
2305.409620461661 False
1252.5292000022473 Fa

402.7582379740643 False
3188.263880404432 False
1008.2646218239561 False
372.76834961687837 False
1410.5047116529122 False
1029.1479501357408 False
3032.437116763934 False
2055.98121700347 False
742.0888822934776 False
894.3724321623167 False
1747.717408937834 False
3719.5736030303297 False
615.0667260126693 False
1866.8238945426265 False
293.0967172906685 False
615.0667260126693 False
1736.2065713298334 False
1917.649669494901 False
1252.5292000022473 False
611.8094140481317 False
1391.2868585218703 False
1917.649669494901 False
677.547565953214 False
2379.719980352572 False
35.31876744604142 False
2671.4617301362678 False
2055.98121700347 False
2454.220754742314 False
2358.0444135337902 False
196.96414390253085 False
1564.3989218075176 False
718.4048260172394 False
464.75582027084783 False
165.42564264373596 False
1286.0747572021346 False
1680.3438469078365 False
1286.0747572021346 False
421.63708332412625 False
1726.169928784403 False
2305.409620461661 False
5145.1506739120605 False

...

790.063758270866 False
1208.6842470016127 False
3035.88965047088 False
5078.086402604525 False
1762.7157452387482 False
1284.0266081273958 False
3188.263880404432 False
1738.6197434441865 False
680.1009865305004 False
1412.5004099985517 False
1564.3989218075176 False
1560.6965072306311 False
1252.5292000022473 False
1543.0820464596918 False
184.82929177822825 False
3253.811979268962 False
222.11236202539658 False
165.42564264373596 False
1109.4464189405026 False
1165.082123274714 False
2404.0030262097616 False
2379.719980352572 False
2671.4617301362678 False
347.03628810350165 False
680.1009865305004 False
775.6073965877113 False
2101.860802740831 False
677.547565953214 False
603.7323771353895 False
3032.437116763934 False
2454.220754742314 False
1284.0266081273958 False
2014.0663522072982 False
603.7323771353895 False
901.6332809629976 False
790.063758270866 False
1868.264564236898 False
3719.5736030303297 False
845.4578544781189 False
79.34763683865913 False
2404.0030262097616 False


896.360028593097 False
1286.0747572021346 False
35.31876744604142 False
245.32384421974166 False
245.32384421974166 False
1412.5004099985517 False
3375.621421088642 False
1866.8238945426265 False
845.4578544781189 False
1736.2065713298334 False
1286.0747572021346 False
2018.4212727523745 False
222.11236202539658 False
1866.8238945426265 False
718.4048260172394 False
347.03628810350165 False
1564.3989218075176 False
1680.3438469078365 False
598.4292101587325 False
5406.096792260173 False
2358.0444135337902 False
293.0967172906685 False
2685.925311341095 False
1680.3438469078365 False
401.0844220603067 False
598.4292101587325 False
3253.811979268962 False
2685.925311341095 False
485.55764138421455 False
2454.220754742314 False
1543.0820464596918 False
3032.437116763934 False
1866.8238945426265 False
790.063758270866 False
165.42564264373596 False
525.4344343253088 False
3459.688932513528 False
3719.5736030303297 False
16.829001448826602 False
1598.0617714531809 False
688.8715080657448 Fa

2012.6714850148664 False
2892.3903801734696 False
775.6073965877113 False
1410.5047116529122 False
1917.649669494901 False
5406.096792260173 False
1286.0747572021346 False
611.8094140481317 False
2055.98121700347 False
196.96414390253085 False
1109.4464189405026 False
196.96414390253085 False
615.0667260126693 False
2014.0663522072982 False
2740.1719613021514 False
894.3724321623167 False
1747.717408937834 False
2358.0444135337902 False
2305.409620461661 False
2892.3903801734696 False
5145.1506739120605 False
3035.88965047088 False
2012.6714850148664 False
1165.4402145426875 False
1564.3989218075176 False
109.26520918770257 False
5406.096792260173 False
421.63708332412625 False
1252.5292000022473 False
611.8094140481317 False
1109.4464189405026 False
165.42564264373596 False
1726.169928784403 False
332.41952604144353 False
165.42564264373596 False
2892.3903801734696 False
3624.470471482032 False
5078.086402604525 False
2671.4617301362678 False
1060.713977222069 False
1008.2646218239561

1866.8238945426265 False
1680.3438469078365 False
1208.6842470016127 False
1165.4402145426875 False
742.0888822934776 False
9.52446486340341 False
3032.437116763934 False
2892.3903801734696 False
896.360028593097 False
9.52446486340341 False
523.7318775970134 False
3459.736460259038 False
2767.863526374832 False
14.328042762917903 False
245.32384421974166 False
2358.0444135337902 False
2404.0030262097616 False
680.1009865305004 False
742.0888822934776 False
1286.0747572021346 False
293.0967172906685 False
2895.6183339852605 False
2671.4617301362678 False
372.76834961687837 False
5078.086402604525 False
2101.860802740831 False
35.31876744604142 False
3188.263880404432 False
2014.0663522072982 False
525.4344343253088 False
742.0888822934776 False
2055.98121700347 False
79.34763683865913 False
35.31876744604142 False
1680.3438469078365 False
485.55764138421455 False
1040.4464338513717 False
1917.649669494901 False
677.547565953214 False
523.7318775970134 False
1680.3438469078365 False
156

KeyboardInterrupt: 

In [92]:
def fitness(populations, numPop, desiredProb):
        #calculate individual fitness fitness of populations
    nestedPopulations = populations
    fitness = []
    totalFitness = 0 #used to sum all the fitnesses
    for i in range(numPop):
        fitness.append(1 / pathLength(nestedPopulations[i]))
        totalFitness += fitness[i]
    
    #print(fitness)
    #find probablilties
    bestSolutionProb = 0
    probOfPopulation = []
    for i in range(numPop):
        probOfPopulation.append(fitness[i]/totalFitness)
        if(probOfPopulation[i]>=desiredProb):
            bestSolutionProb = probOfPopulation[i]
                
    #print(probOfPopulation)

    #pick on of the population
    probDistList = [] #will be used to creat a probabilty distribution of our probOfPopulation
    for i in range(numPop):
        if i == 0:
            probDistList.append(probOfPopulation[i]) #makes the first probabilty calulated the first of the list
        else:
            probDistList.append(probDistList[i-1] + probOfPopulation[i]) #adds up each probability, last value should equal 1
           
    return probDistList, probOfPopulation, bestSolutionProb, fitness
